In [115]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

# All requested packages already installed.

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

# All requested packages already installed.

Libraries imported.


### Using Beautiful Soup for Web Scraping

In [116]:
# import libraries
from urllib.request import urlopen as UReq
from bs4 import BeautifulSoup as soup
# specify the url
urlpage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 


# query the website and return the html to the variable 'page'
UClient = UReq(urlpage)

# Opening up the connection and grabbing the page
page_html=UClient.read()


### Load UClient, turn into soup and get the tables

In [117]:
#
#page_html= open('List_of_postal_codes_of_Canada:_M.html').read()

# parse the html using beautiful soup and store in variable 'page_soup'
page_soup = soup(page_html, 'html.parser')

#Scrape out the required table 
tables2=page_soup.find_all('table',class_="wikitable sortable")



In [118]:
#Using prettify to get the html of the required field
#print(page_soup.prettify())

## Exatracting information from web using Html tags

In [119]:
#Exacted headers for the tables
header=[]
for headers in tables2:
    ths=headers.findAll('th')
    for th in ths:
        header.append(th.text)
header

['Postcode', 'Borough', 'Neighbourhood\n']

In [120]:
#Exacting required text fields from the wiki table 
rowValList = []

for i in range(len(page_soup.findAll('td'))):
    rowVal = page_soup.findAll('td')[i].get_text()
    rowValList.append(rowVal)
#rowValList

In [121]:
#separating the text from the above list into seperate lists
postcode = []
for i in range(0, len(rowValList), 3):
    postcode.append(rowValList[i])
    
borough = []
for i in range(1, len(rowValList), 3):
    borough.append(rowValList[i])
    
neigh = []
for i in range(2, len(rowValList), 3):
    neigh.append(rowValList[i])


### Cleaning up the data to match the requirement

In [122]:
#cleaning the data and creating a data frame for each list above
pc_df = pd.DataFrame(postcode)
pc_df.drop(pc_df.tail(12).index,inplace=True)
#print(pc_df)


b_df = pd.DataFrame(borough)
b_df.drop(b_df.tail(11).index,inplace=True)
#b_df

n_df = pd.DataFrame(neigh)#,columns=neigh)
n_df.drop(n_df.tail(11).index,inplace=True)
#n_df

In [123]:
#concatenating all the dataframes column wise into one dataframe called result
final1 = pd.concat([pc_df, b_df,n_df], axis=1, ignore_index=True)


In [124]:
#adding headers to the dataframe
final1.columns=['Postalcode', 'Borough', 'Neighbourhood']


In [125]:
#Striping out unnecesary text from neighbourhood field and the final dataframe is ready to use

final1['Neighbourhood'] = final1['Neighbourhood'].str.rstrip('\n')



In [126]:
#Just duplicating the dataframe before further cleaning so that the original dataframe will not be effected

final2=final1.copy()


### Processing only rows with assigned Borough

In [127]:
# Get names of indexes for which column Neighbourhood has value Not assigned
indexNames = final1[final1['Borough'] == 'Not assigned' ].index
indexNames 
# Delete these row indexes from dataFrame
final1.drop(indexNames , inplace=True)
final1=final1.reset_index(drop=True)


### Grouping the dataset with PostalCode and Borough

In [140]:
#Grouping more than one neighbourhood with same postal codes and Borough
final1 = final1.groupby(['Postalcode','Borough']).agg(lambda x: ", ".join(x.astype(str))).reset_index()


,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Assigning the 'Not assigned' values in Neighbourhood with the value in Borough

In [141]:
final1.loc[final1.Neighbourhood == 'Not assigned', 'Neighbourhood'] = final1.Borough
final1

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [144]:
final1.rename(columns = {'PostalCode':'Postal Code'}, inplace = True) 
final1.columns

Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')

### Shape of final dataset

In [145]:
final1.shape



(103, 3)

## Getting latitudes and longitudes for each neighbourhood

In [146]:
!wget -q -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

/bin/sh: wget: command not found
Data downloaded!


In [147]:
#reading CSV file which contains latitudes and longitudes
df_geo = pd.read_csv(r'/Users/ShivaNangi/Downloads/Geospatial_Coordinates.csv')


## Merging two dataframes to make a dataframe which contains complete details about the neighborhood

In [148]:
df_lat_long = pd.merge(final1,df_geo, on='Postal Code')
df_lat_long.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Use geopy library to get the latitude and longitude values of Toronto.

In [149]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [150]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_lat_long['Latitude'], df_lat_long['Longitude'], df_lat_long['Borough'], df_lat_long['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Lets explore the neighbourhood in downtown Toronto. Lets segment and cluster only the neighborhoods in downtown Toronto. So let's slice the original dataframe and create a new dataframe of the downtown Toronto.

In [151]:
downtown_data = df_lat_long[df_lat_long['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


### Using geopy library to find the latitude and Longitude of Downtown Toronto

In [152]:
address = '	Downtown Toronto, CA'

geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6563221, -79.3809161.


In [153]:
# create map of Downtown Toronto using latitude and longitude values
map_DToronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Borough'], downtown_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DToronto)  
    
map_DToronto

### Utilizing the Foursquare API to explore the neighborhoods and segment them. Define Foursquare Credentials and Version

In [154]:
CLIENT_ID = 'OQMCUMAYEJL0Z0UI2UCTYXGCOV1FOM1YMDVWSQ3EAS0BAUIQ' # your Foursquare ID
CLIENT_SECRET = 'P3HAPVYCE551JQAW5TRQRJ5GBDS0BIT4SENEU52FLQK0VNXV' # your Foursquare Secret
VERSION = '20191223' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: OQMCUMAYEJL0Z0UI2UCTYXGCOV1FOM1YMDVWSQ3EAS0BAUIQ
CLIENT_SECRET:P3HAPVYCE551JQAW5TRQRJ5GBDS0BIT4SENEU52FLQK0VNXV


#### Let's explore the first neighborhood in our dataframe.

In [155]:
downtown_data.loc[0, 'Neighbourhood']

'Rosedale'

### Getting details about the neighbourhood

In [156]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


### let's get the top 100 venues that are in Rosedale within a radius of 500 meters.

In [157]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=OQMCUMAYEJL0Z0UI2UCTYXGCOV1FOM1YMDVWSQ3EAS0BAUIQ&client_secret=P3HAPVYCE551JQAW5TRQRJ5GBDS0BIT4SENEU52FLQK0VNXV&v=20191223&ll=43.6795626,-79.37752940000001&radius=500&limit=100'

#### Send the GET request and examine the resutls

In [158]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e0123f829ce6a001b06c27e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4aff2d47f964a520743522e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/playground_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e7941735',
         'name': 'Playground',
         'pluralName': 'Playgrounds',
         'primary': True,
         'shortName': 'Playground'}],
       'id': '4aff2d47f964a520743522e3',
       'location': {'address': '38 Scholfield Ave.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Edgar Ave.',
        'distance': 327,
        'formattedAddress': ['38 Scholfield Ave. (at Edgar Ave.)',
         'Toronto ON',
         'Canada'],
        'labeled

#### **get_category_type** function from the Foursquare lab to examine the data in items key

In [159]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [160]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [161]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Exploring nearby neighbourhoods in DownTown

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [162]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [163]:
DownTown_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )


Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Queen's Park


#### Let's check the size of the resulting dataframe

In [164]:
print(DownTown_venues.shape)
DownTown_venues.head()

(1276, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


### Let's check how many venues were returned for each neighborhood

In [165]:
DownTown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",90,90,90,90,90,90
Christie,17,17,17,17,17,17
Church and Wellesley,84,84,84,84,84,84
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


### 3. Analyze Each Neighborhood

In [167]:
# one hot encoding
DownTown_onehot = pd.get_dummies(DownTown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DownTown_onehot['Neighborhood'] = DownTown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DownTown_onehot.columns[-1]] + list(DownTown_onehot.columns[:-1])
DownTown_onehot = DownTown_onehot[fixed_columns]

DownTown_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### And let's examine the new dataframe size.

In [168]:
DownTown_onehot.shape

(1276, 209)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [169]:
DownTown_grouped = DownTown_onehot.groupby('Neighborhood').mean().reset_index()
DownTown_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.00,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.000000,0.010000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.000000,0.0

In [170]:
DownTown_grouped.shape

(18, 209)

#### Let's print each neighborhood along with the top 5 most common venues

In [171]:
num_top_venues = 5

for hood in DownTown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DownTown_grouped[DownTown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2        Steakhouse  0.04
3               Bar  0.04
4  Sushi Restaurant  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1            Café  0.04
2  Farmers Market  0.04
3      Steakhouse  0.04
4          Bakery  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                 venue  freq
0      Airport Service  0.18
1       Airport Lounge  0.12
2      Harbor / Marina  0.06
3                  Bar  0.06
4  Rental Car Location  0.06


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.07
2                Café  0.04
3         Pizza Place  0.04
4  Italian Restaurant  0.04


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.14
1   Italian Restaurant  0.05
2       Sandwich Place  0.05
3  

#### Let's put that into a *pandas* dataframe

In [172]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [173]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DownTown_grouped['Neighborhood']

for ind in np.arange(DownTown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DownTown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Restaurant,Bakery,Salad Place,Asian Restaurant,Sushi Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Café,Cheese Shop,Farmers Market,Bakery,Beer Bar,Steakhouse,Seafood Restaurant,Cocktail Bar,Clothing Store
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Plane,Harbor / Marina,Rental Car Location,Boat or Ferry,Coffee Shop,Sculpture Garden,Bar,Airport Terminal
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Bakery,Pizza Place,Park,Gastropub,Breakfast Spot
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Burger Joint,Café,Ice Cream Shop,Gym / Fitness Center,Fried Chicken Joint,Salad Place


### 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [174]:
# set number of clusters
kclusters = 5

DownTown_grouped_clustering = DownTown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DownTown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 4, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [178]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

DownTown_merged = downtown_data 

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DownTown_merged = DownTown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

DownTown_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1.0,Park,Playground,Trail,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0.0,Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Bakery,Pizza Place,Park,Gastropub,Breakfast Spot
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.0,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Theater,Gym,Hotel,Gastropub,Café
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Theater,Brewery
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Fast Food Restaurant,Café,Restaurant,Bakery,Plaza,Pizza Place


Finally, let's visualize the resulting clusters

In [182]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DownTown_merged['Latitude'], DownTown_merged['Longitude'], DownTown_merged['Neighbourhood'], DownTown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

#### Cluster 1

In [183]:
DownTown_merged.loc[DownTown_merged['Cluster Labels'] == 0, DownTown_merged.columns[[1] + list(range(5, DownTown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0.0,Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Bakery,Pizza Place,Park,Gastropub,Breakfast Spot
2,Downtown Toronto,0.0,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Theater,Gym,Hotel,Gastropub,Café
3,Downtown Toronto,0.0,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Café,Theater,Brewery
4,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Fast Food Restaurant,Café,Restaurant,Bakery,Plaza,Pizza Place
5,Downtown Toronto,0.0,Café,Coffee Shop,Restaurant,Hotel,Clothing Store,Cosmetics Shop,Bakery,Breakfast Spot,Beer Bar,Italian Restaurant
6,Downtown Toronto,0.0,Coffee Shop,Café,Cheese Shop,Farmers Market,Bakery,Beer Bar,Steakhouse,Seafood Restaurant,Cocktail Bar,Clothing Store
7,Downtown Toronto,0.0,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Burger Joint,Café,Ice Cream Shop,Gym / Fitness Center,Fried Chicken Joint,Salad Place
8,Downtown Toronto,0.0,Coffee Shop,Café,Bar,Steakhouse,Restaurant,Bakery,Salad Place,Asian Restaurant,Sushi Restaurant,Burger Joint
9,Downtown Toronto,0.0,Coffee Shop,Aquarium,Café,Hotel,Sporting Goods Shop,Fried Chicken Joint,Brewery,Italian Restaurant,Scenic Lookout,Restaurant
10,Downtown Toronto,0.0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Bar,Gastropub,Seafood Restaurant,American Restaurant,Bakery


In [184]:
DownTown_merged.loc[DownTown_merged['Cluster Labels'] == 1, DownTown_merged.columns[[1] + list(range(5, DownTown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1.0,Park,Playground,Trail,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


In [185]:
DownTown_merged.loc[DownTown_merged['Cluster Labels'] == 2, DownTown_merged.columns[[1] + list(range(5, DownTown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2.0,Airport Service,Airport Lounge,Plane,Harbor / Marina,Rental Car Location,Boat or Ferry,Coffee Shop,Sculpture Garden,Bar,Airport Terminal


In [186]:
DownTown_merged.loc[DownTown_merged['Cluster Labels'] == 3, DownTown_merged.columns[[1] + list(range(5, DownTown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,3.0,Café,Japanese Restaurant,Bar,Bakery,Restaurant,Bookstore,Sandwich Place,Chinese Restaurant,Dessert Shop,Massage Studio


In [187]:
DownTown_merged.loc[DownTown_merged['Cluster Labels'] == 4, DownTown_merged.columns[[1] + list(range(5, DownTown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,4.0,Grocery Store,Café,Park,Athletics & Sports,Diner,Baby Store,Restaurant,Candy Store,Italian Restaurant,Nightclub
